In [2]:
import numpy as np
import pandas as pd
import math
import glob
from astroquery.ipac.irsa import Irsa
from astropy.coordinates import SkyCoord
from astropy import units as u
from astroquery.simbad import Simbad
from astropy.table import Table

In [3]:
#import the data frames. rewrote some of the columns to make them the way i wanted.
#testing
gsample = pd.read_csv('../Data/galaxy_names_reduced_homogeneous_resolved_unique_with_10ks_exposure_cut.csv')
#print(gsample)
gsample = pd.DataFrame(gsample['Galaxy_Name_Reduced'][:])

gsample = gsample.rename(columns={'Galaxy_Name_Reduced': 'source_id'})
gsample['source_id']
#print(gsample)
#gsample.to_csv('output1.csv', index=True)

oldsample = pd.read_csv('../Data/ocatResult_Modified.csv')
#oldsample
#gsample

In [4]:
# I want only a specific few columns just to make it simpler

columns = ['RA','Dec','Gname']
name_and_coords = oldsample[columns]
name_and_coords

,RA,Dec,Gname
0,201.365,-43.019,NGC 5128
1,186.550,12.950,MESSIER 086
2,184.741,47.303,MESSIER 106
3,184.741,47.303,MESSIER 106
4,202.468,47.198,MESSIER 051
...,...,...,...
720,190.533,32.542,NGC 4631
721,179.484,55.454,NGC 3998
722,179.484,55.454,NGC 3998
723,201.365,-43.019,NGC 5128


In [5]:
gsample['source_id']

0         NGC 5128
1      MESSIER 086
2      MESSIER 106
3      MESSIER 051
4      MESSIER 082
          ...     
116       NGC 4302
117       NGC 1792
118          IC342
119       NGC 3287
120    MESSIER 096
Name: source_id, Length: 121, dtype: object

In [6]:

# this block is useful for doing simbad queries but does not really help. some interesting functions like "to list"

sample_list = gsample['source_id'].tolist()
#print(sample_list)
#Set simbad settings to retrieve Ra and Dec
simbad_settings = Simbad()
#simbad_settings.add_votable_feilds("coordinates")
#testing to make sure querying simbad works
test= Simbad.query_object("M1")

#querying simbad using the list running over each name in the list and pulling out all the information on it.
#i specifically wanted to see the RA and DEC but i'll get many other things from it and that's ok
myresult= []

for galaxy in sample_list:
    result = Simbad.query_object(galaxy)
    myresult.append(result)

#myresult

In [7]:
# vs code

In [8]:
# the code that Katie helped me with. Not something that i should be practicing since apparently you should not
# should not use for loops with pandas since the reason it exists is to eliminate the need for them
# nonetheless, this uses names from the list of samples above and goes into "name_and_coords"(which is a variable for
# a column reduced csv file that Anthony gave us with all the galaxies in the sample and had way too many duplicates)
# and finds the location (.loc) for any name that matches what is in the small csv file with the big csv file and appends it
# as you can see it reduces the number somewhat cuz of the 10ks cut, but there are a lot of duplicates

targets = sample_list
columns = name_and_coords.columns
#create empty data frame
target_data = pd.DataFrame(columns=columns)
for target in targets:
    row = name_and_coords.loc[name_and_coords['Gname']==target]
    target_data = target_data._append(row)


target_data


/var/folders/nd/77pj43rx7wbcbp0wx2qjq2140000gp/T/ipykernel_61121/303889620.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  target_data = target_data._append(row)


,RA,Dec,Gname
0,201.365,-43.019,NGC 5128
47,201.365,-43.019,NGC 5128
104,201.370,-43.017,NGC 5128
131,201.370,-43.017,NGC 5128
272,201.330,-43.045,NGC 5128
...,...,...,...
702,161.691,11.820,MESSIER 096
703,161.691,11.820,MESSIER 096
704,161.691,11.820,MESSIER 096
706,161.691,11.820,MESSIER 096


In [9]:
# removes the duplicates
target_data.drop_duplicates(subset = ['Gname'])

,RA,Dec,Gname
0,201.365,-43.019,NGC 5128
1,186.550,12.950,MESSIER 086
2,184.741,47.303,MESSIER 106
4,202.468,47.198,MESSIER 051
6,148.963,69.679,MESSIER 082
...,...,...,...
578,185.427,14.598,NGC 4302
595,76.310,-37.981,NGC 1792
645,56.702,68.096,IC342
694,158.702,21.662,NGC 3287


In [10]:
#wanted to rename the columns again cuz i was fighting with the document upload feature on WISE telescope

target_data = target_data.rename(columns={'Gname': 'source_id'})
target_data = target_data.rename(columns={'RA': 'ra'})
target_data = target_data.rename(columns={'Dec': 'dec'})

target_data = target_data.drop_duplicates(subset = ['source_id'])

In [11]:
target_data

,ra,dec,source_id
0,201.365,-43.019,NGC 5128
1,186.550,12.950,MESSIER 086
2,184.741,47.303,MESSIER 106
4,202.468,47.198,MESSIER 051
6,148.963,69.679,MESSIER 082
...,...,...,...
578,185.427,14.598,NGC 4302
595,76.310,-37.981,NGC 1792
645,56.702,68.096,IC342
694,158.702,21.662,NGC 3287


In [12]:
targetnames = target_data

In [13]:
#write each row as separated by |
#with open('../Data/file_ascii.txt', 'w', encoding = 'ascii', errors = 'ignore') as f:
   # for index, row in target_data:
     #   row_string = '|'.join(map(str, row.values)) + '\n'
       # f.write(row_string)

# still fighting but i'll leave it here just in case   
    
columns1 = ['ra','dec']
target_data = target_data[columns1]
target_data

,ra,dec
0,201.365,-43.019
1,186.550,12.950
2,184.741,47.303
4,202.468,47.198
6,148.963,69.679
...,...,...
578,185.427,14.598
595,76.310,-37.981
645,56.702,68.096
694,158.702,21.662


In [14]:
#target_data.to_csv('final1.csv', index=False, sep = ' ')

In [15]:

#a nice thing i found on stack overflow for creating a .tbl file! writing it into one of the formats that WISE only accepts

data_out = Table.from_pandas(target_data)
output_path = '../Data/data_out.tbl'

# Write the table in IPAC format
data_out.write(output_path, format='ipac')

OSError: File ../Data/data_out.tbl already exists. If you mean to replace it then use the argument "overwrite=True".

In [19]:
#data = np.genfromtxt('../Data/table_irsa_catalog_search_results.tbl')

# found out that you can change the type of document you can download from WISE which helped a lot (csv files are goated)
data = pd.read_csv('table_irsa_catalog_search_results.csv')
data

#THE DESCRIPTIONS FOR EACH COLUMN IN THE DATA TABLE CAN BE SEEN IN THE DOWNLOADED FILE!! i think you have a version of it as a tbl which is a little more legible.

,cntr_01,dist_x,pang_x,ra_01,dec_01,designation,ra,dec,sigra,sigdec,...,ph_qual,moon_lev,w1nm,w1m,w2nm,w2m,w3nm,w3m,w4nm,w4m
0,1,0.830776,162.673944,201.365,-43.019,J132527.62-430109.1,201.365094,-43.019220,0.0263,0.0252,...,AAAA,0,46,46,46,46,24,24,24,24
1,3,5.022737,-43.454463,184.741,47.303,J121857.50+471814.4,184.739585,47.304013,0.0301,0.0302,...,AAAA,0,34,34,34,34,19,19,19,19
2,4,6.295556,-65.412743,202.468,47.198,J132951.75+471155.4,202.465660,47.198728,0.0304,0.0294,...,AAAA,0,32,32,32,32,18,18,18,18
3,5,4.934001,60.594118,148.963,69.679,J095551.94+694046.8,148.966438,69.679673,0.0014,0.0014,...,CABA,0,39,39,39,39,19,19,19,19
4,5,9.488868,61.783241,148.963,69.679,J095552.72+694048.8,148.969688,69.680246,0.0033,0.0032,...,BACA,0,32,40,40,40,18,20,18,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,115,1.168653,-43.519281,188.613,2.188,J123427.06+021117.6,188.612776,2.188235,0.0300,0.0291,...,AAAA,2211,23,23,22,22,13,13,13,13
110,116,2.389462,-83.980790,185.427,14.598,J122142.31+143553.0,185.426318,14.598070,0.0321,0.0323,...,AAAA,111,25,25,24,24,13,13,13,13
111,117,1.456962,3.629091,76.310,-37.981,J050514.40-375850.1,76.310033,-37.980596,0.0292,0.0288,...,AAAA,0,57,57,57,57,30,30,30,30
112,118,1.037780,-27.456965,56.702,68.096,J034648.39+680546.5,56.701644,68.096256,0.0304,0.0279,...,AAAA,11,25,25,25,25,10,10,11,11


In [ ]:

# for some reason the "merge" function wants the columns in both dataframes to be named the exact same
# so this makes things simpler
gcolumns= ['ra_01','dec_01']
simple_data = data[gcolumns]
simple_data = simple_data.rename(columns={'ra_01': 'ra'})
simple_data = simple_data.rename(columns={'dec_01': 'dec'})
simple_data

,ra,dec
0,201.365,-43.019
1,184.741,47.303
2,202.468,47.198
3,148.963,69.679
4,148.963,69.679
...,...,...
109,188.613,2.188
110,185.427,14.598
111,76.310,-37.981
112,56.702,68.096


In [ ]:

#useful code from katie. not sure what "inner" means, maybe that it doesnt prioritize one dataframe over another.
merged_data = pd.merge(df1, df2, on='the column you want to match', how='inner')
merged_data = merged_data.drop_duplicates()
#merged_data

In [ ]:
#using merge to find which of the 114 galaxies found in the WISE telescope are in the list that Anthony gave us. Used a "left merge"
df_all = targetnames.merge(simple_data.drop_duplicates(), on=['ra','dec'], 
                   how='left', indicator=True)
print(df_all[0:122])

          ra     dec    source_id     _merge
0    201.365 -43.019     NGC 5128       both
1    186.550  12.950  MESSIER 086  left_only
2    184.741  47.303  MESSIER 106       both
3    202.468  47.198  MESSIER 051       both
4    148.963  69.679  MESSIER 082       both
..       ...     ...          ...        ...
115  185.427  14.598     NGC 4302       both
116   76.310 -37.981     NGC 1792       both
117   56.702  68.096        IC342       both
118  158.702  21.662     NGC 3287  left_only
119  161.691  11.820  MESSIER 096       both

[120 rows x 4 columns]


In [ ]:
df_all['_merge'] == True
#don't remember how to do pull out just the one's that are left_only.

0      False
1      False
2      False
3      False
4      False
       ...  
115    False
116    False
117    False
118    False
119    False
Name: _merge, Length: 120, dtype: bool

In [ ]:
# below here I tried to use the OG method Katie and i found but i couldnt get it to work

In [ ]:
gcolumns= ['ra','dec']
simple_data = data[gcolumns]
simple_data

,ra,dec
0,201.365094,-43.019220
1,184.739585,47.304013
2,202.465660,47.198728
3,148.966438,69.679673
4,148.969688,69.680246
...,...,...
109,188.612776,2.188235
110,185.426318,14.598070
111,76.310033,-37.980596
112,56.701644,68.096256


In [ ]:
targets = simple_data
gcolumns = targetnames.columns
#create empty data frame
missing = pd.DataFrame(columns=gcolumns)
for target in targets:
    row = targetnames.loc[targetnames['ra']==target]
    missing = missing._append(row)

missing
#nothing is showing up because it is not the exact value. you need a partial match, that is something that this does not seem to be able to do.

,ra,dec,source_id
